In [29]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import seaborn
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from pycaret.classification import *
from pycaret.classification import evaluate_model
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import selenium
from selenium import webdriver
import random
from selenium.webdriver.common.by import By
from sklearn.preprocessing import MinMaxScaler

In [69]:
fot_df = pd.read_csv('current_stats.csv', index_col = [0])

In [311]:
def winner(x):
    if x == 0:
        return 0
    elif x > 0:
        return 1
    else:
        return 2

In [51]:
df = pd.read_csv('og_half.csv', index_col = [0])
df = df[half_cols]

In [642]:
df = pd.read_csv('fulltime.csv', index_col = [0])
df = df[df['Expected goals (xG)'].notna()]
df = df.drop(['url', 'xG penalty'], axis = 1)
# df['league'] = pd.factorize(df['league'])[0]
df['target'] = df['score'].apply(winner)

In [22]:
df

,Corners,Fouls Committed,Goals,Offsides,Pass Completion,Possesion,Red Cards,Shot Attempts,Shots Blocked,Total Passes,Total Shots on Target,Total xG,Yellow Cards,target
0,-4.0,-1.0,-1.0,0.0,0.149,0.248,0.0,-6.0,-3.0,115.0,-1.0,-0.806711,0.0,2
1,0.0,0.0,0.0,2.0,0.157,0.316,0.0,-2.0,-4.0,178.0,0.0,-0.323895,0.0,1
2,1.0,0.0,0.0,0.0,-0.097,-0.064,0.0,3.0,2.0,-35.0,3.0,-0.247529,-1.0,2
3,0.0,1.0,1.0,-1.0,0.023,-0.008,0.0,0.0,2.0,-5.0,-2.0,0.722193,1.0,1
4,0.0,0.0,-1.0,0.0,-0.114,-0.360,0.0,-1.0,1.0,-198.0,-2.0,-0.525874,2.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,6.0,0.0,0.0,0.0,0.134,0.256,0.0,12.0,3.0,120.0,4.0,0.593374,-1.0,2
678,-4.0,3.0,-1.0,1.0,-0.180,-0.320,0.0,-13.0,-3.0,-153.0,-6.0,-1.904855,-1.0,2
679,-6.0,1.0,1.0,0.0,-0.149,-0.292,0.0,-5.0,-2.0,-121.0,-3.0,-0.460209,1.0,1
680,-1.0,0.0,2.0,0.0,0.144,0.330,0.0,5.0,1.0,196.0,0.0,1.188723,0.0,1


In [43]:
# df = df[scraped_cols]
scaler = MinMaxScaler()
df[df.columns[:-1]] = scaler.fit_transform(df[df.columns[:-1]])

In [52]:
half_cols = ['Corners', 'Goals', 'Offsides','Pass Completion', 'Possesion', 'Red Cards', 'Shot Attempts',
       'Shots Blocked', 'Total Passes', 'Total Shots on Target', 'Total xG', 'Yellow Cards', 'target']

In [67]:
scraped_cols = ['Corners', 'Fouls committed', 'score', 'Offsides', 'Accurate passes percentage',
       'posession','Red cards','Total shots', 'Blocks','Passes', 'Shots on target','Expected goals (xG)',
       'Yellow cards','url']

In [5]:
train, test = train_test_split(df, test_size=0.2)

In [10]:
print(train.shape)
print(test.shape)

(545, 14)
(137, 14)


In [53]:
grid = setup(data=df, target='target', feature_selection = True)#, remove_multicollinearity = True)

,Description,Value
0,session_id,4864
1,Target,target
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(682, 13)"
5,Missing Values,False
6,Numeric Features,12
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [54]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7296,0.8637,0.5963,0.6723,0.6739,0.5400,0.5657,0.0590
ridge,Ridge Classifier,0.7172,0.0000,0.5773,0.5991,0.6488,0.5152,0.5450,0.0050
lda,Linear Discriminant Analysis,0.7088,0.8539,0.5739,0.6081,0.6491,0.5059,0.5305,0.0050
et,Extra Trees Classifier,0.7047,0.8387,0.5927,0.6582,0.6694,0.5061,0.5191,0.1790
rf,Random Forest Classifier,0.7025,0.8369,0.6002,0.6722,0.6748,0.5059,0.5176,0.1900
nb,Naive Bayes,0.6983,0.8354,0.5874,0.6567,0.6623,0.4975,0.5126,0.0050
qda,Quadratic Discriminant Analysis,0.6857,0.8367,0.5820,0.6512,0.6611,0.4814,0.4898,0.0060
gbc,Gradient Boosting Classifier,0.6857,0.8295,0.5783,0.6473,0.6560,0.4812,0.4922,0.1200
xgboost,Extreme Gradient Boosting,0.6669,0.8170,0.5634,0.6317,0.6424,0.4525,0.4604,0.0620
lightgbm,Light Gradient Boosting Machine,0.6606,0.8241,0.5635,0.6359,0.6418,0.4432,0.4506,0.0290


In [56]:
lr = create_model('lr')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7292,0.8106,0.5952,0.5938,0.6532,0.5398,0.5728
1,0.7917,0.8950,0.6693,0.8276,0.7368,0.6499,0.6776
2,0.7708,0.8856,0.6178,0.6438,0.7010,0.6060,0.6328
3,0.7708,0.8924,0.6444,0.8109,0.7276,0.6112,0.6337
4,0.6875,0.8679,0.5370,0.6002,0.6245,0.4537,0.4953
5,0.6250,0.7951,0.5383,0.6013,0.5932,0.3881,0.4064
6,0.7083,0.9011,0.5888,0.7540,0.6692,0.4978,0.5172
7,0.7021,0.8687,0.5624,0.6169,0.6550,0.4989,0.5122
8,0.7447,0.8650,0.5927,0.6368,0.6834,0.5573,0.5838


In [55]:
tuned = tune_model(best)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7292,0.8106,0.5952,0.5938,0.6532,0.5398,0.5728
1,0.7917,0.8950,0.6693,0.8276,0.7368,0.6499,0.6776
2,0.7708,0.8856,0.6178,0.6438,0.7010,0.6060,0.6328
3,0.7708,0.8916,0.6444,0.8109,0.7276,0.6112,0.6337
4,0.6875,0.8679,0.5370,0.6002,0.6245,0.4537,0.4953
5,0.6250,0.7946,0.5383,0.6013,0.5932,0.3881,0.4064
6,0.7083,0.9005,0.5888,0.7540,0.6692,0.4978,0.5172
7,0.7021,0.8698,0.5624,0.6169,0.6550,0.4989,0.5122
8,0.7447,0.8650,0.5927,0.6368,0.6834,0.5573,0.5838


In [57]:
evaluate_model(tuned)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [40]:
tuned

NameError: name 'LogisticRegression' is not defined

In [229]:
def get_stats(url):
    driver = webdriver.Chrome()
    driver.implicitly_wait(random.randint(0, 6))
    driver.get(url)
    ht = driver.find_elements(By.XPATH, '//span[@class="css-8ngw4i-bottomRow"]')[1].text
    if ht != 'Half-Time':
        print('not halftime just yet ... sleeping')
        ht = driver.find_elements(By.XPATH, '//span[@class="css-8ngw4i-bottomRow"]')[1].text
        if ht != 'Half-Time':
            print(f'time: {ht}')
    button = driver.find_elements(By.XPATH, '(//button[@class="css-12l9xvx-ExpandButton e7i6a0l0"])[2]')[0]
    button.click()
    div1 = driver.find_elements(By.XPATH, '//li[@class="css-radwzz-Stat e683amr6"]')
    t = {k[1]: [k[0], k[2]] for k in [i.split('\n') for i in [x for x in [d.text for d in div1]
          if x not in ['Shots','Passes','Expected goals (xG)','Discipline','Defence','Duels']]]}
    score = driver.find_element(By.XPATH, '//span[@class="css-bw7eig-topRow"]').get_attribute('textContent').split(' ')
    t['score'] = [score[0], score[2]]
    posession = driver.find_element(By.XPATH, '//div[@class="css-7s52se-PossessionWheel e683amr3"]').text.split('\n')
    t['posession'] = [posession[1].replace('%',''), posession[2].replace('%','')]
    game_stats = {}
    for name, stats in t.items():
        if '(' in ''.join(stats):
            s = stats[0].split(' ') + stats[1].split(' ')
            if len(s) == 4:
                s = [i.replace('(', '').replace(')','').replace('%','') for i in s]
                game_stats[name] = [s[0], s[2]]
                game_stats[name + ' percentage'] = [int(s[1])/100, int(s[3])/100]
            else:
                s = [i for i in s if '%' not in i]
                game_stats[name] = s
        else:
            game_stats[name] = stats
    game_stats = {k: float(v[0]) - float(v[1]) for k, v in game_stats.items()}
    game_stats['match_name'] = url.split('/')[-1]
    game_stats['half_time'] = ht
    game_stats['url'] = url
    driver.quit()
    return game_stats

In [230]:
s = get_stats('https://www.fotmob.com/match/3900958/matchfacts/manchester-united-vs-liverpool')

not halftime just yet ... sleeping
time: 47:19


In [493]:
live = pd.DataFrame(s, index = [0])
live = live[scraped_cols]
# live = live[live.columns[:-1]]

In [97]:
fot_df = fot_df[scraped_cols].reset_index(drop = True)
# fot_df['pred'] = lr.predict(fot_df)

KeyError: "['url'] not in index"

In [56]:
s_test = fot_df[scraped_cols]
s_test.columns = test.columns[:-1]

In [58]:
preds = predict_model(tuned, raw_score = True).tail(30)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.7171,0.8684,0.5977,0.7225,0.6639,0.5197,0.5417


In [59]:
preds

,Shot Attempts,Total xG,Possesion,Goals,Total Passes,Corners,Total Shots on Target,Offsides,Pass Completion,target,Label,Score_0,Score_1,Score_2
175,-9.0,-0.420262,-0.416,0.0,-215.0,-3.0,-4.0,0.0,-0.198,2,2,0.1847,0.1500,0.6653
176,1.0,-0.574675,0.400,0.0,218.0,1.0,2.0,0.0,0.199,1,1,0.2517,0.5516,0.1966
177,13.0,0.965932,0.404,0.0,181.0,6.0,7.0,1.0,0.216,1,1,0.3468,0.5737,0.0795
178,-2.0,0.097664,-0.434,-1.0,-266.0,-3.0,-1.0,-2.0,-0.098,2,2,0.1773,0.0510,0.7717
179,-10.0,-0.962742,-0.362,0.0,-204.0,-3.0,-6.0,0.0,-0.160,2,2,0.1340,0.1200,0.7460
180,5.0,0.583291,-0.174,0.0,-93.0,-3.0,0.0,0.0,-0.092,2,2,0.2315,0.3812,0.3873
181,2.0,0.020170,0.204,0.0,101.0,2.0,2.0,0.0,0.049,1,1,0.3073,0.4727,0.2200
182,5.0,1.033353,0.362,0.0,171.0,2.0,3.0,-1.0,0.214,1,1,0.2794,0.6502,0.0704
183,1.0,0.862428,-0.418,0.0,-191.0,0.0,0.0,-3.0,-0.148,0,0,0.4064,0.2919,0.3017
184,-4.0,-0.227625,-0.016,1.0,-9.0,-1.0,-1.0,-3.0,-0.006,1,1,0.2298,0.6834,0.0868


In [60]:
tuned.predict_proba([[1.0,
 -6.0,
 0.0,
 -1.0,
 -0.1699999999999999,
 -20.0,
 0.0,
 1.0,
 -1.0,
 -82.0

ValueError: X has 13 features per sample; expecting 9